In [ ]:
# check Pillow version
import PIL
print(PIL.PILLOW_VERSION)

5.3.0


In [ ]:
# we need pillow version of 5.3.0
# we will uninstall the older version first
!pip uninstall -y Pillow

# install the new one
!pip install Pillow==5.3.0

# import the new one
import PIL
print(PIL.PILLOW_VERSION)
# this should print 5.3.0. If it doesn't, then restart your runtime:
# Menu > Runtime > Restart Runtime

Uninstalling Pillow-5.3.0:
  Successfully uninstalled Pillow-5.3.0
  Using cached https://files.pythonhosted.org/packages/62/94/5430ebaa83f91cc7a9f687ff5238e26164a779cca2ef9903232268b0a318/Pillow-5.3.0-cp36-cp36m-manylinux1_x86_64.whl
5.3.0


In [ ]:
! pip3 install torch torchvision

In [ ]:
import os

directory = '/content/results/'
if not os.path.exists(directory):
  os.makedirs(directory)

In [ ]:
import random

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data

import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils

In [ ]:
'''
Note:
  use Conv2d (https://pytorch.org/docs/stable/nn.html#conv2d)

Note:
  nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
'''
from typing import Optional

class Discriminator(nn.Module):
    def __init__(self, 
                 nz:int, 
                 ndf:int, 
                 nc: int = 1, 
                 ngpu: Optional[int] = None):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input:  b x (nc   ) x 28 x 28

            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # output: b x (ndf  ) x 14 x 14

            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # output: b x (ndf*2) x 7 x 7

            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # output: b x (ndf*4) x 3 x 3
            
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # output: b x (ndf*8) x 1 x 1
            
            nn.Conv2d(ndf * 8, 1, 1, 1, 0, bias=False),
            # output: b x 1 x 1 x 1
            
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)
      
'''
Note: 
  use ConvTranspose2d (https://pytorch.org/docs/stable/nn.html#torch.nn.ConvTranspose2d)

Note:
  torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1)
'''
class Generator(nn.Module):
    def __init__(self, 
                 nz:int, 
                 ngf:int, 
                 nc: int = 1, 
                 ngpu: Optional[int] = None):
        super(Generator, self).__init__()
        self.ngpu = ngpu if ngpu else 1
        self.main = nn.Sequential(
            # input:  b x nz x 1 x 1
            
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # output: b x (ngf*8) x 4 x 4
            
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # output: b x (ngf*4) x 8 x 8
            
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 2, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # output: b x (ngf*2) x 14 x 14
            
            nn.ConvTranspose2d(ngf * 2, nc, 4, 2, 1, bias=False),
            # output: b x (nc   ) x 28 x 28 

            nn.Tanh()
            # output: b x (nc   ) x 28 x 28 
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output

In [ ]:
def weights_init(m: nn.Module):
    '''
    custom weights initialization called on G and D
    '''
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [ ]:
# Paths to saved model
saved_G, saved_D = None, None

# Number of GPUs
ngpu = 1

# Batch size
batch_size = 128

# How many worker to use in dataloader
num_workers = 4

# Number of channels in image from dataset
nc = 1

In [ ]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print("Running on %s" % device)

Running on cuda:0


In [ ]:
# Create data
data = datasets.MNIST('data', download=True,
                      transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.5,), (0.5,)),
                       ]))

# Setup efficient dataloading
dataloader = torch.utils.data.DataLoader(
    data, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [ ]:
# Model parameters

# latent vector size
nz = 100

# the number of filters in the discriminator
ndf = 28

# the number of filters in the generator
ngf = 28

In [ ]:
netD.cuda()
netG.cuda()

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 224, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(224, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(224, 112, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(112, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(112, 56, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2), bias=False)
    (7): BatchNorm2d(56, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(56, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): Tanh()
  )
)

In [ ]:
netG = Generator(nz=nz, ngf=ngf, nc = nc, ngpu = ngpu).to(device)
netG.apply(weights_init)
netG.to(device)
if saved_G is not None:
    netG.load_state_dict(torch.load(saved_G))

print(netG)

netD = Discriminator(nz=nz, ndf=ndf, nc = nc, ngpu = ngpu).to(device)
netD.apply(weights_init)
netD.to(device)
if saved_D is not None:
    netD.load_state_dict(torch.load(saved_D))
print(netD)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 224, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(224, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(224, 112, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(112, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(112, 56, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2), bias=False)
    (7): BatchNorm2d(56, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(56, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): Tanh()
  )
)
Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 28, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(28, 56, kernel_size=(4, 4), stride=(2, 2), padd

In [ ]:
# Training parameters
niter = 10
lr = 0.0002
beta1 = 0.5

In [ ]:
# an alternative implementation of GAN objective
criterion = nn.BCELoss()

fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

for epoch in range(1, niter + 1):
    for i, (real, _) in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        real = real.to(device)
        
        # train with real
        netD.zero_grad()
        batch_size = real.size(0)

        output = netD(real)
        D_x = output.mean().item()
        
        label = torch.full((batch_size,), real_label, device=device)
        errD_real = criterion(output, label)
        errD_real.backward()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)

        output = netD(fake.detach())
        D_G_z1 = output.mean().item()
        
        label.fill_(fake_label)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        errD = errD_real + errD_fake
        
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        
        output = netD(fake)
       
        label.fill_(real_label)  # fake labels are real for generator cost
        errG = criterion(output, label)
        errG.backward()
        
        D_G_z2 = output.mean().item()
        optimizerG.step()
        
        if i % 10 == 0:
          print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
                % (epoch, niter, i, len(dataloader),
                   errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        if i % 100 == 0:
            print('[%d/%d][%d/%d] Saving samples!' 
                  % (epoch, niter, i, len(dataloader)))

            vutils.save_image(real,
                    '%s/samples_real.png' % directory,
                    normalize=True)

            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),
                    '%s/samples_fake_epoch_%03d_batch_%03d.png' % (directory, epoch, i),
                    normalize=True)

    # do checkpointing
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (directory, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (directory, epoch))

[1/10][0/469] Loss_D: 1.4072 Loss_G: 0.6584 D(x): 0.5308 D(G(z)): 0.5312 / 0.5201
[1/10][0/469] Saving samples!
[1/10][10/469] Loss_D: 1.3814 Loss_G: 0.7005 D(x): 0.5090 D(G(z)): 0.5040 / 0.4976
[1/10][20/469] Loss_D: 1.3726 Loss_G: 0.7134 D(x): 0.5052 D(G(z)): 0.4966 / 0.4910
[1/10][30/469] Loss_D: 1.3680 Loss_G: 0.7197 D(x): 0.5052 D(G(z)): 0.4938 / 0.4881
[1/10][40/469] Loss_D: 1.3531 Loss_G: 0.7282 D(x): 0.5085 D(G(z)): 0.4900 / 0.4836
[1/10][50/469] Loss_D: 1.3374 Loss_G: 0.7354 D(x): 0.5135 D(G(z)): 0.4872 / 0.4802
[1/10][60/469] Loss_D: 1.3278 Loss_G: 0.7512 D(x): 0.5121 D(G(z)): 0.4804 / 0.4728
[1/10][70/469] Loss_D: 1.3104 Loss_G: 0.7647 D(x): 0.5152 D(G(z)): 0.4745 / 0.4665
[1/10][80/469] Loss_D: 1.2870 Loss_G: 0.7817 D(x): 0.5207 D(G(z)): 0.4671 / 0.4590
[1/10][90/469] Loss_D: 1.2675 Loss_G: 0.7950 D(x): 0.5265 D(G(z)): 0.4623 / 0.4531
[1/10][100/469] Loss_D: 1.2227 Loss_G: 0.8145 D(x): 0.5431 D(G(z)): 0.4549 / 0.4444
[1/10][100/469] Saving samples!
[1/10][110/469] Loss_D: 1

In [ ]:
# Save images from training data
import imageio
import glob

img_path = '/content/results/'
img_tmp = 'samples_fake_*.png'

images = [
    imageio.imread(f) for f in sorted(glob.glob(f"{img_path}/{img_tmp}"))
]

imageio.mimsave(f"{img_path}/training.gif", images, duration=0.2)